In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import numpy as np
import time
import sys

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

ncolor = 1
nclass = 10
width  = 28
height = 28
ntrain = len(x_train)
ntest  = len(x_test)

x_train = x_train/255.0
x_train = np.reshape(x_train, (ntrain, -1))
x_test = x_test/255.0
x_test = np.reshape(x_test, (ntest, -1))

labels = np.zeros((ntrain, nclass))
labels[np.arange(ntrain), y_train] = 1
y_train = labels

labels = np.zeros((ntest, nclass))
labels[np.arange(ntest), y_test] = 1
y_test = labels

print("Number of Training Data: ", ntrain)
print("Number of Test Data: ", ntest)
print("Number of class: ", nclass)

Number of Training Data:  60000
Number of Test Data:  10000
Number of class:  10


In [3]:
class classification_model(object):
    def __init__(self, img_w, img_h, n_class, lr=1e-4):

        # Placeholder Input
        x = tf.placeholder(tf.float32, [None, img_w*img_h])           # (batch, height, width, channel)
        y_ = tf.placeholder(tf.float32, [None, n_class])            # input y
        img = tf.reshape(x, [-1, img_w, img_h, 1])
        dropout_rate = tf.placeholder(tf.float32)

        # Network
        conv1 = tf.layers.conv2d(img, 32, 5, 1, 'same', activation=tf.nn.relu)
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2,)
        conv2 = tf.layers.conv2d(pool1, 64, 5, 1, 'same', activation=tf.nn.relu)
        pool2 = tf.layers.max_pooling2d(conv2, 2, 2)
        pool2f = tf.reshape(pool2, [-1, 7*7*64])
        fc1 = tf.layers.dense(pool2f, 1024, tf.nn.relu)
        drop1 = tf.layers.dropout(inputs=fc1, rate=dropout_rate)
        output = tf.layers.dense(drop1, n_class)
        y = tf.contrib.layers.softmax(output)

        # Optimizer
        loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
        train_op = tf.train.AdamOptimizer(lr).minimize(loss)

        # Variables
        self.x = x
        self.y = y
        self.y_ = y_
        self.loss = loss
        self.accuracy = accuracy
        self.train_op = train_op
        self.dropout_rate = dropout_rate


In [4]:
def classification(data, epochs, batch_size, img_w, img_h, n_class, dropout=0.5, lr=1e-4):     
    x_train, y_train, x_test, y_test = data

    # Create a model first
    cnn = classification_model(img_w, img_h, n_class)
    runs = int(len(x_train) / batch_size)
    train_order = np.arange(len(x_train))

    # Training
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        print("Training Iteration Started: ")
        bgn_time = time.time()
        for epoch in range(epochs):
            random.shuffle(train_order)

            for i in range(runs):
                bgn = i * batch_size
                end = min((bgn + batch_size, len(x_train)-1))
                idx = train_order[bgn:end]

                sess.run(cnn.train_op, {cnn.x:x_train[idx], cnn.y_:y_train[idx], cnn.dropout_rate:dropout})
                if (i%250 == 0) or (i == runs-1):
                    acc = sess.run(cnn.accuracy, {cnn.x:x_test, cnn.y_:y_test, cnn.dropout_rate:0.0})
                    print("\r  epoch:{0} iters:{1}/{2} acc:{3}        ".format(epoch, i, runs, acc))

        acc, loss = sess.run([cnn.accuracy, cnn.loss], {cnn.x:x_test, cnn.y_:y_test, cnn.dropout_rate:0.0})
        print("\r  Labels: ", len(x_train), " Epochs: ", epochs, " Acc: ", acc, " Cross Entropy: ", loss)
        print("  Time Lapse: ", time.time() - bgn_time)

In [5]:
def ranker_random(batch, cnn, sess=None):
    return np.random.randint(10, size=len(batch))

def ranker_max_min(batch, cnn, sess):
    pred_y = sess.run(cnn.y, {cnn.x:batch, cnn.dropout_rate:0.0})
    ranked = np.max(pred_y, axis=1) - np.min(pred_y, axis=1)
    return ranked

def select_active_batch(cnn, x_train, lookup_list, counter, batch_size, sess, ranker):
    reset = 0
    if counter * 100 > len(x_train)-1:
        reset = 1
        lookup_list = random.shuffle(lookup_list)
        counter = 0

    bgn = counter * 100
    end = min(bgn + 100, len(x_train)-1)
    batch_idx = lookup_list[bgn:end]
    batch = x_train[batch_idx]
    counter = counter + 1

    #get the network scores of selected data
    ranked        = ranker(batch, cnn, sess)
    ranked_scores = np.column_stack((batch_idx, ranked))
    sorted_scores = ranked_scores[np.argsort(ranked_scores[:,1])]

    #select examples based on their scores. we only pick the first {batch_size}th of data
    return_size = min(batch_size, len(sorted_scores))
    selected = sorted_scores[:return_size, 0]
    return [selected.astype(int).tolist(), lookup_list, counter]

In [6]:
def select_active_data(data, iters, batch_size, img_w, img_h, n_class, ranker, dropout=0.5, lr=1e-4):
    x_train, y_train, x_test, y_test = data    

    counter = 0
    chosen = []
    lookup_list = np.arange(len(x_train))
    cnn = classification_model(img_w, img_h, n_class)

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        for i in range(iters):
            batch_idx, lookup_list, counter = select_active_batch(cnn, x_train, lookup_list, counter, batch_size, sess, ranker)
            sess.run(cnn.train_op, {cnn.x:x_train[batch_idx], cnn.y_:y_train[batch_idx], cnn.dropout_rate:dropout})
            chosen = chosen + batch_idx

            if (i%10 == 0) or (i == iters-1):
                print("\r  Iters:{0}/{1}       ".format(i, iters), end="")

        acc, loss = sess.run([cnn.accuracy, cnn.loss], {cnn.x:x_test, cnn.y_:y_test, cnn.dropout_rate:0.0})
        print("\n  Labels: ", len(x_train), " Iters: ", iters, " Acc: ", acc, " Cross Entropy: ", loss)
        
    return chosen


In [7]:
lr = 0.001
dropout = 0.5

iters       = 500
select_size = 10
train_epochs        = 5
train_batch_size    = 256
data = [x_train, y_train, x_test, y_test]

# Normal Learning
print("Select Normal Learning Dataset: ")
normal = select_active_data(data, iters, select_size, width, height, nclass, ranker_random, dropout, lr)
normal_data = [x_train[normal], y_train[normal], x_test, y_test]
classification(normal_data, train_epochs, train_batch_size, width, height, nclass, dropout, lr)

# Active Learning
print("\nSelect Active Learning Dataset: ")
active = select_active_data(data, iters, select_size, width, height, nclass, ranker_max_min, dropout, lr)
active_data = [x_train[active], y_train[active], x_test, y_test]
classification(active_data, train_epochs, train_batch_size, width, height, nclass, dropout, lr)

Select Normal Learning Dataset: 
  Iters:499/500       
  Labels:  60000  Iters:  500  Acc:  0.9287  Cross Entropy:  0.24258901
Training Iteration Started: 
  epoch:0 iters:0/19 acc:0.20409999787807465        
  epoch:0 iters:18/19 acc:0.7117000222206116        
  epoch:1 iters:0/19 acc:0.7242000102996826        
  epoch:1 iters:18/19 acc:0.7922999858856201        
  epoch:2 iters:0/19 acc:0.7856000065803528        
  epoch:2 iters:18/19 acc:0.8596000075340271        
  epoch:3 iters:0/19 acc:0.8600000143051147        
  epoch:3 iters:18/19 acc:0.8889999985694885        
  epoch:4 iters:0/19 acc:0.8910999894142151        
  epoch:4 iters:18/19 acc:0.9013000130653381        
  Labels:  5000  Epochs:  5  Acc:  0.9013  Cross Entropy:  0.3610995
  Time Lapse:  27.712556838989258

Select Active Learning Dataset: 
  Iters:499/500       
  Labels:  60000  Iters:  500  Acc:  0.9632  Cross Entropy:  0.15527913
Training Iteration Started: 
  epoch:0 iters:0/19 acc:0.1111999973654747        
  ep